# 3 - Problema de investimentos em projetos

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
projetos = 6
periodos = 4
I = [i for i in range(projetos)]
J = [j for j in range(periodos)]

In [3]:
lucro = [32.4, 35.8, 17.75, 14.8, 18.2, 12.35]
recursos = [60, 70, 35, 20]

In [4]:
desembolso = [[10.5, 14.4, 2.2, 2.4],[8.3, 12.6, 9.5, 3.1],[10.2, 14.2, 5.6, 4.2],
              [7.2, 10.5, 7.5, 5.0],[12.3, 10.1, 8.3, 6.3],[9.2, 7.8, 6.9, 5.1]]
custo_total = [29.5, 33.5, 34.2, 30.2, 37.0, 29.0]

### Caso A

In [5]:
# Modelo
m = gp.Model("Investimento-a")

# Variáveis de decisão
x = m.addVars (I, vtype = GRB.BINARY, obj = lucro, name = "x")

# Restrições
# Respeitar a disponibilidade de recursos
m.addConstrs(gp.quicksum(x[i]*desembolso[i][j] for i in I) <= recursos[j] for j in J)

m.ModelSense = GRB.MAXIMIZE
m.update()
m.write("Modelo_Investimento-a.lp")
m.optimize()

# Saída...
for i in range(projetos):
    print(f"Projeto {i+1} -> Executar (1-sim, 0-não): {x[i].x}")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-07
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 6 columns and 24 nonzeros
Model fingerprint: 0xc4be9124
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [2e+00, 1e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 7e+01]
Found heuristic solution: objective 113.1000000
Presolve removed 3 rows and 2 columns
Presolve time: 0.03s
Presolved: 1 rows, 4 columns, 4 nonzeros
Variable types: 0 continuous, 4 integer (4 binary)

Root relaxation: cutoff, 0 iterations, 0.01 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.08 seconds (0.00 work units)
Thread count was 12 (of 12 available processo

### Caso B

In [9]:
# Modelo
m = gp.Model("Investimento-b")

# Variáveis de decisão
x = m.addVars (I, vtype = GRB.BINARY, obj = lucro, name = "x")

# Restrições
# Respeitar a disponibilidade de recursos
m.addConstrs(gp.quicksum(x[i]*desembolso[i][j] for i in I) <= recursos[j] for j in J)

# Projetos não conduzidos simultaneamente
m.addConstr(x[1] + x[3] <= 1, "c02")

# Projetos conduzidos simultaneamente
m.addConstr(x[0] - x[5] == 0, "c03")

m.ModelSense = GRB.MAXIMIZE
m.update()
m.write("Modelo_Investimento-b.lp")
m.optimize()

# Saída...
for i in range(projetos):
    print(f"Projeto {i+1} -> Executar (1-sim, 0-não): {x[i].x}")


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 6 columns and 28 nonzeros
Model fingerprint: 0xdcee6905
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective 71.7500000
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 3: 98.75 98.75 71.75 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.875000000000e+01, best bound 9.875000000000e+01, gap 0.0000%
Projeto 1 -> Executar (1-sim, 0-não): 1.0
Projeto 2 

### Caso C

In [7]:
# Modelo
m = gp.Model("Investimento-c")

# Variáveis de decisão
x = m.addVars (I, vtype = GRB.BINARY, obj = lucro, name = "x")
s = m.addVars (I, vtype = GRB.CONTINUOUS, lb = 0.0, obj = 0.0, name = "s")

# Restrições
# Respeitar a disponibilidade de recursos - Ano 0
m.addConstr(s[0] + gp.quicksum(x[i]*desembolso[i][0] for i in I) == recursos[0])

# Respeitar a disponibilidade de recursos - Anos 1, 2 e 3
m.addConstrs(s[j] + gp.quicksum(x[i]*desembolso[i][j] for i in I) == recursos[j] + s[j-1] for j in J if j>0)

m.ModelSense = GRB.MAXIMIZE
m.update()
m.write("Modelo_Investimento-c.lp")
m.optimize()

# Saída...
for i in range(projetos):
    print(f"Projeto {i+1} -> Executar (1-sim, 0-não): {x[i].x}")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 12 columns and 31 nonzeros
Model fingerprint: 0x89d9c235
Variable types: 6 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 7e+01]
Presolve removed 0 rows and 3 columns
Presolve time: 0.00s
Presolved: 4 rows, 9 columns, 29 nonzeros
Variable types: 0 continuous, 9 integer (6 binary)
Found heuristic solution: objective 118.9500000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 118.95 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.189500000000e+02, best bound 1.189500000000e+02, gap 0.0000%
Projet

### Caso D

In [6]:
# Modelo
m = gp.Model("Investimento-d")

# Variáveis de decisão
x = m.addVars (I, vtype = GRB.CONTINUOUS, lb = 0.0, ub = 1.0, obj = lucro, name = "x")

# Restrições
# Respeitar a disponibilidade de recursos
m.addConstrs(gp.quicksum(x[i]*desembolso[i][j] for i in I) <= recursos[j] for j in J)

m.ModelSense = GRB.MAXIMIZE
m.update()
m.write("Modelo_Investimento-d.lp")
m.optimize()

# Saída...
for i in range(projetos):
    print(f"Projeto {i+1} -> Executar (1-sim, 0-não): {x[i].x}")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 6 columns and 24 nonzeros
Model fingerprint: 0xd6a4783e
Coefficient statistics:
  Matrix range     [2e+00, 1e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 7e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1545455e+02   9.727273e+00   0.000000e+00      0s
       3    1.1606111e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.160611111e+02
Projeto 1 -> Executar (1-sim, 0-não): 1.0
Projeto 2 -> Executar (1-sim, 0-não): 1.0
Projeto 3 -> Executar (1-sim, 0-não): 1.0
Projeto 4 -> Execu

### Caso E

In [16]:
# Nesta versão do problema foi admitido que qualquer fração do projeto possa ser realizada em qualquer ano,
# e que o desembolso em cada ano é proporcional à fração realizada do projeto.

# Modelo
m = gp.Model("Investimento-e")

# Variáveis de decisão
x = m.addVars (I, vtype = GRB.BINARY, obj = lucro, name = "x")
y = m.addVars (I, J, vtype = GRB.CONTINUOUS, lb = 0.0, ub = 1.0, obj = 0.0, name = "y")

# Restrições
# Respeitar a disponibilidade de recursos
m.addConstrs(gp.quicksum(y[i,j]*custo_total[i] for i in I) <= recursos[j] for j in J)

# As frações devem ser igual a 0 ou 1
m.addConstrs(gp.quicksum(y[i,j] for j in J) == x[i] for i in I)

m.ModelSense = GRB.MAXIMIZE
m.update()
m.write("Modelo_Investimento-e.lp")
m.optimize()

# Saída...
for i in range(projetos):
    print(f"Projeto {i+1} -> Executar (1-sim, 0-não): {x[i].x}")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10 rows, 30 columns and 54 nonzeros
Model fingerprint: 0xd4e92efd
Variable types: 24 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 7e+01]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 10 rows, 30 columns, 54 nonzeros
Variable types: 24 continuous, 6 integer (6 binary)
Found heuristic solution: objective 113.1000000

Root relaxation: objective 1.277228e+02, 11 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  127.72276   